In [1]:
import torch
import torch.nn as nn
import model_pt
import importlib
importlib.reload(model_pt)



/home/rsim/anaconda3/envs/fedrec/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'model_pt' from '/home/rsim/FedNewsRec/code/model_pt.py'>

In [2]:
import tensorflow as tf
import keras
from keras.layers import *
from keras.models import Model

def tfDenseTest(dim1,dim2):
    vecs_input = Input(shape=(dim1,dim2),dtype='float32')
    user_att = Dense(100,activation='tanh', kernel_initializer= 'random_uniform', bias_initializer= 'random_uniform')(vecs_input)
    model = Model(vecs_input,user_att)
    return model


/home/rsim/anaconda3/envs/fedrec/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rsim/anaconda3/envs/fedrec/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rsim/anaconda3/envs/fedrec/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rsim/anaconda3/envs/fedrec/lib/python3.7/site-

In [3]:
dim1 = 5
dim2 = 10
x = torch.randn((2, dim1, dim2))

pool = model_pt.AttentivePooling(dim1, dim2)

#pool(x).shape


pool(x)


user_vecs torch.Size([2, 5, 10])
dense1 torch.Size([2, 5, 200])
flatten torch.Size([2, 5])
softmax torch.Size([2, 5])
dot torch.Size([2, 10])


tensor([[ 0.1211, -0.4455, -1.1638,  0.5727, -0.9729,  0.5442,  0.4008,  0.3014,
          0.2522, -0.6350],
        [-1.5661,  0.8745,  0.7691, -0.6407, -0.2056, -0.4551,  0.1764,  0.3775,
          1.0522, -0.2276]], grad_fn=<ViewBackward0>)

In [4]:
import tensorflow as tf
import keras
from keras.layers import *
from keras.models import Model


def tfAttentivePooling(dim1,dim2, name=None):
    vecs_input = Input(shape=(dim1,dim2),dtype='float32')
    user_vecs =Dropout(0.2)(vecs_input)
    user_att = Dense(200,activation='tanh', kernel_initializer= 'random_uniform', bias_initializer= 'random_uniform')(user_vecs)
    user_att = keras.layers.Flatten()(Dense(1, kernel_initializer= 'random_uniform', bias_initializer= 'random_uniform')(user_att))
    user_att = Activation('softmax')(user_att)    
    user_vec = keras.layers.Dot((1,1))([user_vecs,user_att])
    model = Model(vecs_input,user_vec, name=name)
    return model

def testDot(dim1, dim2):
    input = Input(shape=(dim1,dim2), dtype='float32')
    output = keras.layers.Dot((2,2))([input, input])
    return Model(input, output)


In [5]:
t = testDot(2,3)
x_raw = [[[1,2,3],[4,5,6]]]
x_tf = tf.convert_to_tensor(x_raw)
t(x_tf).eval(session=tf.compat.v1.Session())
x_pt = torch.tensor(x_raw)


2022-06-04 17:43:42.619010: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-04 17:43:42.641240: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2445435000 Hz
2022-06-04 17:43:42.642282: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5600c3ac0880 executing computations on platform Host. Devices:
2022-06-04 17:43:42.642314: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-06-04 17:43:42.991708: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0001:00:00.0
totalMemory: 14.56GiB freeMemory: 14.37GiB
2022-06-04 17:43:42.991742: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1512] Adding visible gpu devices: 0
2022-06-04 17:43:42.993132: I tensorflow/core/common_runtime/gpu/

In [7]:
import tensorflow as tf
import keras
from keras.layers import *
from keras.models import Model
import torch

dim1 = 5
dim2 = 10
x = torch.randn((2, dim1, dim2))


def tfAttentivePooling(dim1,dim2, name=None):
    vecs_input = Input(shape=(dim1,dim2),dtype='float32')
    user_vecs =Dropout(0.2)(vecs_input)
    user_att = Dense(200,activation='tanh', kernel_initializer= 'random_uniform', bias_initializer= 'random_uniform')(user_vecs)
    user_att = keras.layers.Flatten()(Dense(1, kernel_initializer= 'random_uniform', bias_initializer= 'random_uniform')(user_att))
    user_att = Activation('softmax')(user_att)    
    user_vec = keras.layers.Dot((1,1))([user_vecs,user_att])
    model = Model(vecs_input,user_vec, name=name)
    return model

pool = tfAttentivePooling(dim1, dim2)

def print_model(m):
    for layer in m.layers:
        try:
            print(layer.name, layer.output_shape, [w.shape for w in layer.get_weights()]) 
        except:
            print(layer.name, 'output0', layer.get_output_shape_at(0))

print_model(pool)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_tf = tf.convert_to_tensor(x.numpy())
    pool(x_tf).eval()


input_3 (None, 5, 10) []
dropout_2 (None, 5, 10) []
dense_3 (None, 5, 200) [(10, 200), (200,)]
dense_4 (None, 5, 1) [(200, 1), (1,)]
flatten_2 (None, 5) []
activation_2 (None, 5) []
dot_3 (None, 10) []


2022-06-04 17:44:41.944008: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1512] Adding visible gpu devices: 0
2022-06-04 17:44:41.944058: I tensorflow/core/common_runtime/gpu/gpu_device.cc:984] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-06-04 17:44:41.944063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990]      0 
2022-06-04 17:44:41.944066: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1003] 0:   N 
2022-06-04 17:44:41.944115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 13983 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5)


In [15]:
pool_pt = model_pt.AttentivePooling(dim1, dim2)

for layer in pool.layers:
    print(layer.name)
    if layer.name=='dense_3':
        weights = layer.get_weights()
        pool_pt.dense.weight.data = torch.tensor(weights[0]).transpose(0,1)
        pool_pt.dense.bias.data = torch.tensor(weights[1])
    elif layer.name == 'dense_4':
        weights = layer.get_weights()
        pool_pt.dense2.weight.data = torch.tensor(weights[0]).transpose(0,1)
        pool_pt.dense2.bias.data = torch.tensor(weights[1])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_tf = tf.convert_to_tensor(x.numpy())
    y_tf = pool(x_tf).eval()
    print(y_tf.shape)
    with torch.no_grad():
        pool_pt.eval()
        print(y_tf-pool_pt(x).detach().numpy())
    #print(pool_pt(x).numpy())

input_3
dropout_2
dense_3
dense_4
flatten_2
activation_2
dot_3
(2, 10)
user_vecs torch.Size([2, 5, 10])
dense1 torch.Size([2, 5, 200])
flatten torch.Size([2, 5])
softmax torch.Size([2, 5])
dot torch.Size([2, 10])
[[ 0.07939512  0.00312686  0.00488412  0.09010908  0.44272405  0.07363436
  -0.1563558  -0.05927446  0.05221535  0.0496828 ]
 [ 0.26080972  0.12523896  0.07913249  0.07507315  0.03815147  0.13829823
   0.0458912   0.30603275 -0.07538399 -0.12466186]]


2022-06-04 17:50:03.660200: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1512] Adding visible gpu devices: 0
2022-06-04 17:50:03.660245: I tensorflow/core/common_runtime/gpu/gpu_device.cc:984] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-06-04 17:50:03.660250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990]      0 
2022-06-04 17:50:03.660254: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1003] 0:   N 
2022-06-04 17:50:03.660330: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 13983 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5)


In [9]:
from models import Attention

def get_doc_encoder():
    sentence_input = Input(shape=(30,300), dtype='float32')
    droped_vecs = Dropout(0.2)(sentence_input)

    l_cnnt = Conv1D(400,3,activation='relu')(droped_vecs)
    l_cnnt = Dropout(0.2)(l_cnnt)
    l_cnnt = Attention(20,20)([l_cnnt,l_cnnt,l_cnnt])
    l_cnnt = keras.layers.Activation('relu')(l_cnnt)
    
    droped_rep = Dropout(0.2)(l_cnnt)
    title_vec = tfAttentivePooling(30,400)(droped_rep)
    sentEncodert = Model(sentence_input, title_vec)
    return sentEncodert

d = get_doc_encoder()
print_model(d)


input_3 (None, 30, 300)
dropout_2 (None, 30, 300)
conv1d_1 (None, 28, 400)
dropout_3 (None, 28, 400)
attention_1 (None, 28, 400)
activation_2 (None, 28, 400)
dropout_4 (None, 28, 400)
model_3 (None, 400)


In [10]:
class Permute(nn.Module):
    def __init__(self, *dims):
        super(Permute, self).__init__()
        self.dims = dims
    
    def forward(self, x):
        return x.permute(*self.dims)

class SwapTrailingAxes(nn.Module):
    def __init__(self):
        super(SwapTrailingAxes, self).__init__()
        
    def forward(self, x):        
        return x.mT

class DocEncoder(nn.Module):
    def __init__(self):        
        super(DocEncoder,self).__init__()
        self.phase1 = nn.Sequential(
            nn.Dropout(0.2),
            SwapTrailingAxes(),            
            nn.Conv1d(300,400,3),
            nn.ReLU(),
            nn.Dropout(0.2),
            SwapTrailingAxes()
        )
        self.attention = nn.MultiheadAttention(400,20)
        self.phase2 = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.2),
            AttentivePooling(30,400)
        )
    
    def forward(self, x):
        l_cnnt = self.phase1(x)
        print('doc_encoder:phase1',l_cnnt.shape)
        l_cnnt, attention_weights = self.attention(l_cnnt, l_cnnt, l_cnnt)
        print('doc_encoder:attention', l_cnnt.shape)
        result = self.phase2(l_cnnt)
        print('doc_encoder:phase2', result.shape)
        return result


In [11]:
d_pt = DocEncoder()
dim1 = 30
dim2 = 300
x = torch.randn((2, dim1, dim2))

d_pt.forward(x)

doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phase2 torch.Size([2, 400])


tensor([[2.5350e-02, 6.0881e-02, 1.0347e-02, 1.4556e-02, 1.1262e-01, 1.6728e-02,
         5.2783e-02, 9.6941e-03, 7.8505e-02, 4.0878e-02, 3.2404e-02, 9.0031e-02,
         1.0173e-01, 9.2766e-02, 1.6397e-01, 7.4023e-02, 5.1577e-02, 5.8122e-02,
         4.1079e-02, 1.9458e-02, 3.9305e-02, 6.1552e-02, 1.8461e-02, 4.3711e-02,
         5.3473e-02, 2.3207e-02, 0.0000e+00, 1.2538e-01, 2.2131e-02, 8.8368e-02,
         2.5095e-02, 1.6292e-01, 1.6799e-01, 6.5956e-03, 0.0000e+00, 1.2744e-01,
         3.7824e-02, 2.8322e-01, 0.0000e+00, 7.3282e-02, 5.2872e-02, 7.2333e-04,
         7.5436e-02, 1.6102e-01, 9.2541e-02, 1.9440e-03, 7.9293e-02, 3.1075e-02,
         9.0353e-03, 3.7613e-02, 1.7275e-02, 7.7970e-02, 3.8281e-02, 1.6803e-02,
         5.2499e-02, 6.6982e-02, 1.8000e-02, 1.1917e-01, 1.8935e-01, 3.4093e-02,
         9.9588e-02, 2.3916e-02, 5.6093e-03, 6.4216e-02, 1.0940e-02, 1.2334e-01,
         8.2217e-02, 1.4190e-01, 8.0195e-02, 3.8005e-02, 7.1861e-02, 4.2199e-02,
         3.4656e-02, 6.0225e

In [12]:

def get_user_encoder():
    news_vecs_input = Input(shape=(50,400), dtype='float32', name='news_vecs_input')
    
    news_vecs = Dropout(0.2, name='news_vecs')(news_vecs_input)
    gru_input = keras.layers.Lambda(lambda x:x[:,-15:,:])(news_vecs)
    vec1 = GRU(400)(gru_input)
    vecs2 = Attention(20,20, name='vecs2')([news_vecs]*3)
    vec2 = tfAttentivePooling(50,400)(vecs2)

    user_vecs2 = Attention(20,20, name='attention_user_vecs2')([news_vecs_input]*3)
    user_vecs2 = Dropout(0.2, name='dropout_user_vecs2')(user_vecs2)
    user_vec2 = tfAttentivePooling(50,400, name='pooling_user_vec2')(user_vecs2)
    user_vec2 = keras.layers.Reshape((1,400))(user_vec2)
        
    user_vecs1 = Lambda(lambda x:x[:,-20:,:], name='tail20_user_vec1')(news_vecs_input)
    user_vec1 = GRU(400, name="gru_user_vec1")(user_vecs1)
    user_vec1 = keras.layers.Reshape((1,400))(user_vec1)

    user_vecs = keras.layers.Concatenate(axis=-2)([user_vec1,user_vec2])
    vec = tfAttentivePooling(2,400, name='vec_pool')(user_vecs)
        
    sentEncodert = Model(news_vecs_input, vec, name='user_encoder')
    return sentEncodert

u = get_user_encoder()
print_model(u)

news_vecs_input (None, 50, 400)
attention_user_vecs2 (None, 50, 400)
tail20_user_vec1 (None, 20, 400)
dropout_user_vecs2 (None, 50, 400)
gru_user_vec1 (None, 400)
pooling_user_vec2 (None, 400)
reshape_2 (None, 1, 400)
reshape_1 (None, 1, 400)
concatenate_1 (None, 2, 400)
vec_pool (None, 400)


In [13]:
class VecTail(nn.Module):
    def __init__(self, n):
        super(VecTail, self).__init__()
        self.n = n

    def forward(self, x):
        return x[:,-self.n:,:]

class UserEncoder(nn.Module):
    def __init__(self):        
        super(UserEncoder,self).__init__()
        # news_vecs_input = Input(shape=(50,400), dtype='float32')
        #self.dropout1 = nn.Dropout(0.2)
        #self.tail = VecTail(15)
        #self.gru = nn.GRU(400, 400)
        #self.attention = nn.MultiheadAttention(400, 20)
        #self.pool = AttentivePooling(50, 400)
        self.attention2 = nn.MultiheadAttention(400, 20)
        self.dropout2 = nn.Dropout(0.2)
        self.pool2 = AttentivePooling(50, 400)
        self.tail2 = VecTail(20)
        self.gru2 = nn.GRU(400,400, batch_first=True)
        self.pool3 = AttentivePooling(2, 400)

    def forward(self, news_vecs_input):    
        #news_vecs =self.dropout1(news_vecs_input)
        #gru_input = self.tail(news_vecs)
        #vec1 = self.gru(gru_input)
        #vecs2 = self.attention(*[news_vecs]*3)
        #vec2 = self.pool(vecs2)
        print('news_vecs_input', news_vecs_input.shape)
        user_vecs2, _u_weights = self.attention2(*[news_vecs_input]*3)
        user_vecs2 = self.dropout2(user_vecs2)
        user_vec2 = self.pool2(user_vecs2)
        print('pool2_user_vec2', user_vec2.shape)
        #user_vec2 = keras.layers.Reshape((1,400))(user_vec2)
        #user_vec2 = user_vec2.unsqueeze(1)

        user_vecs1 = self.tail2(news_vecs_input)
        print('tail2_user_vecs1', user_vecs1.shape)
        user_vec1, _u_hidden = self.gru2(user_vecs1)
        print('gru2_user_vec1', user_vec1.shape)
        user_vec1 = user_vec1[:, -1, :]
        #user_vec1 = keras.layers.Reshape((1,400))(user_vec1)
        #user_vec1 = user_vec1.unsqueeze(1)
        
        user_vecs = torch.stack([user_vec1, user_vec2], dim=1) #keras.layers.Concatenate(axis=-2)([user_vec1,user_vec2])
        print(user_vecs.shape)
        vec = self.pool3(user_vecs)
        print(vec.shape)
        return vec
        


    


In [14]:
u_pt = UserEncoder()
dim1 = 50
dim2 = 400
x = torch.randn((2, dim1, dim2))

u_pt.forward(x)

news_vecs_input torch.Size([2, 50, 400])
pool2_user_vec2 torch.Size([2, 400])
tail2_user_vecs1 torch.Size([2, 20, 400])
gru2_user_vec1 torch.Size([2, 20, 400])
torch.Size([2, 2, 400])
torch.Size([2, 400])


tensor([[ 2.6806e-02,  1.3265e-02, -1.6694e-01,  2.1280e-02,  1.8720e-01,
         -7.0635e-03, -3.2158e-01,  1.3890e-02, -7.7261e-02, -7.7432e-03,
         -9.5752e-02, -8.2957e-02,  2.1183e-01, -2.6020e-02, -7.8267e-03,
         -5.9289e-02,  1.1708e-02,  1.6052e-02, -3.6814e-01,  6.8838e-02,
          6.1980e-03,  1.1181e-01,  2.0714e-01,  1.4299e-02,  2.5372e-02,
          2.4576e-03, -2.4605e-01,  1.6293e-01, -1.0431e-02,  0.0000e+00,
         -1.6275e-01,  2.9142e-02,  2.1689e-02,  1.5156e-01,  2.6457e-01,
         -7.2167e-02,  5.7367e-02, -1.2071e-01, -4.0362e-02, -1.9674e-01,
          6.6915e-02, -2.7688e-01,  2.2345e-01, -9.4654e-02, -3.6159e-01,
          1.3844e-02,  6.0253e-02, -2.1889e-01,  2.8887e-01, -1.4123e-01,
          3.2666e-01, -3.0967e-02,  1.6158e-01, -6.5765e-03,  6.3870e-03,
          1.4175e-01,  1.9355e-01,  1.1138e-01, -1.9907e-02,  1.3860e-01,
         -5.9715e-02,  3.5732e-01, -3.3400e-02,  1.0295e-01, -1.7454e-02,
         -3.1576e-01, -6.5066e-02, -1.

In [15]:
from keras.optimizers import SGD
from keras.layers import TimeDistributed as tfTimeDistributed
npratio = 4
def get_model(lr,delta,title_word_embedding_matrix,optimizer_name='sgd'):
    doc_encoder = get_doc_encoder()
    user_encoder = get_user_encoder()
    
    title_word_embedding_layer = Embedding(title_word_embedding_matrix.shape[0], 300, weights=[title_word_embedding_matrix],trainable=False)
    
    click_title = Input(shape=(50,30),dtype='int32')
    can_title = Input(shape=(1+npratio,30),dtype='int32')
    
    click_word_vecs = title_word_embedding_layer(click_title)
    can_word_vecs = title_word_embedding_layer(can_title)
    
    click_vecs = tfTimeDistributed(doc_encoder, name='td_clicks')(click_word_vecs)
    can_vecs = tfTimeDistributed(doc_encoder, name='td_can')(can_word_vecs)
    
    user_vec = user_encoder(click_vecs)
    
    scores = keras.layers.Dot(axes=-1)([user_vec,can_vecs]) #(batch_size,1+1,) 
    logits = keras.layers.Activation(keras.activations.softmax, name='recommend')(scores)     
    
    optimizer = None
    if optimizer_name == 'sgd':
        optimizer = SGD(lr=lr, clipvalue=delta)
    elif optimizer_name == 'adam':
        optimizer = Adam(lr=lr, clipvalue=delta)
    assert optimizer

    model = Model([can_title,click_title],logits, name='model') # max prob_click_positive
    model.compile(loss=['categorical_crossentropy'],
                  optimizer=optimizer, 
                  metrics=['acc'])
    
    news_input = Input(shape=(30,),dtype='int32')
    news_word_vecs = title_word_embedding_layer(news_input)
    news_vec = doc_encoder(news_word_vecs)
    news_encoder = Model(news_input,news_vec, name='news_encoder')
    
    return model, doc_encoder, user_encoder, news_encoder

mtx = np.random.uniform(size=(10,300)).astype('float32')
#mtx_tf = tf.convert_to_tensor(mtx)
#print(mtx_tf.shape[0])
model, doc_encoder, user_encoder, news_encoder = get_model(0.1, 0.0, mtx)

print('model')
print_model(model)
print('******')
print('doc')
print_model(doc_encoder)
print('******')
print('user')
print_model(user_encoder)
print('******')
print('news')
print_model(news_encoder)

2022-05-15 21:29:11.323378: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1512] Adding visible gpu devices: 0
2022-05-15 21:29:11.323436: I tensorflow/core/common_runtime/gpu/gpu_device.cc:984] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-05-15 21:29:11.323441: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990]      0 
2022-05-15 21:29:11.323445: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1003] 0:   N 
2022-05-15 21:29:11.323532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 13983 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0001:00:00.0, compute capability: 7.5)


model
input_13 (None, 50, 30)
embedding_1 output0 (None, 50, 30, 300)
input_14 (None, 5, 30)
td_clicks (None, 50, 400)
user_encoder (None, 400)
td_can (None, 5, 400)
dot_11 (None, 5)
recommend (None, 5)
******
doc
input_8 (None, 30, 300)
dropout_9 (None, 30, 300)
conv1d_2 (None, 28, 400)
dropout_10 (None, 28, 400)
attention_2 (None, 28, 400)
activation_7 (None, 28, 400)
dropout_11 (None, 28, 400)
model_6 (None, 400)
******
user
news_vecs_input (None, 50, 400)
attention_user_vecs2 (None, 50, 400)
tail20_user_vec1 (None, 20, 400)
dropout_user_vecs2 (None, 50, 400)
gru_user_vec1 (None, 400)
pooling_user_vec2 (None, 400)
reshape_4 (None, 1, 400)
reshape_3 (None, 1, 400)
concatenate_2 (None, 2, 400)
vec_pool (None, 400)
******
news
input_15 (None, 30)
embedding_1 output0 (None, 50, 30, 300)
model_7 (None, 400)


In [16]:

# todo: verify
class TimeDistributed(nn.Module):    
    def __init__(self, module): #, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        # self.batch_first = batch_first

    def forward(self, x):
        print('TimeDist_x',x.size())
        if len(x.size()) <= 2:
            return self.module(x)

        output = torch.tensor([])
        for i in range(x.size(1)):
          output_t = self.module(x[:, i, :, :])
          output_t  = output_t.unsqueeze(1)
          output = torch.cat((output, output_t ), 1)
        print('TimeDist_output', output.size())
        return output
        # # Squash samples and timesteps into a single axis
        # x_reshape = x.contiguous().view(x.size(0), -1, x.size(-1))  # (samples * timesteps, input_size)
        #print('TimeDist_x_reshape',x_reshape.shape)
        # y = self.module(x_reshape)
        # print('TimeDist_y', y.shape)
        # # We have to reshape Y
        # if self.batch_first:
        #     y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        # else:
        #    y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)
        # print('TimeDist_y_reshape',y.size())
        #return y

class FedNewsRec(nn.Module):
    def __init__(self, title_word_embedding_matrix):
        super(FedNewsRec, self).__init__()
        self.doc_encoder = DocEncoder() 
        self.user_encoder = UserEncoder()
        # TODO: should this embedding matrix be frozen?
        self.title_word_embedding_layer = nn.Embedding.from_pretrained(torch.tensor(title_word_embedding_matrix), freeze=False)
    
        # click_title = Input(shape=(50,30),dtype='int32')
        # can_title = Input(shape=(1+npratio,30),dtype='int32')
    
        self.softmax = nn.Softmax(dim=1)
        self.click_td = TimeDistributed(self.doc_encoder) #, batch_first=True)
        self.can_td = TimeDistributed(self.doc_encoder) #, batch_first=True)
        
    def forward(self, click_title, can_title, news_input):
        
        click_word_vecs = self.title_word_embedding_layer(click_title)
        print('click',click_word_vecs.shape)
        can_word_vecs = self.title_word_embedding_layer(can_title)
        print('can', can_word_vecs.shape)
        click_vecs = self.click_td(click_word_vecs)
        print('click_vecs (None, 50, 400)', click_vecs.shape)
        can_vecs = self.can_td(can_word_vecs)
        print('can_vecs (None, 5, 400)', can_vecs.shape)
    
        user_vec = self.user_encoder(click_vecs)        
        print('user_vec (None, 400)', user_vec.shape)
        # TODO verify
        scores = torch.einsum('ijk,ik->ij',  can_vecs, user_vec)
        print('scores  (None, 5)', scores.shape)
        logits = self.softmax(scores)     
        print('logits  (None, 5)', logits.shape)
        
        news_word_vecs = self.title_word_embedding_layer(news_input)
        news_vec = self.doc_encoder(news_word_vecs)
        
        print('user_vec', user_vec.shape)
        print('news_vec', news_vec.shape)
        return logits, user_vec, news_vec

In [17]:
dim1 = 50
dim2 = 400
click = torch.randint(10, (2,50,30))
can = torch.randint(10, (2,5,30))
news = torch.randint(10, (2,30))

x = torch.randn((2, dim1, dim2))

f = FedNewsRec(mtx)
#print(f.click_td)
f.forward(click,can,news)


click torch.Size([2, 50, 30, 300])
can torch.Size([2, 5, 30, 300])
TimeDist_x torch.Size([2, 50, 30, 300])
doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phase2 torch.Size([2, 400])
doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phase2 torch.Size([2, 400])
doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phase2 torch.Size([2, 400])
doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phase2 torch.Size([2, 400])
doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phase2 torch.Size([2, 400])
doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phase2 torch.Size([2, 400])
doc_encoder:phase1 torch.Size([2, 28, 400])
doc_encoder:attention torch.Size([2, 28, 400])
doc_encoder:phas

(tensor([[0.2006, 0.2008, 0.1978, 0.2008, 0.1999],
         [0.2023, 0.2005, 0.1980, 0.2003, 0.1989]], grad_fn=<SoftmaxBackward0>),
 tensor([[-6.3956e-03, -9.1935e-03, -4.7749e-03,  3.4982e-02,  1.1907e-02,
          -5.6878e-02, -2.7117e-02, -4.9031e-04,  7.8465e-03, -8.6454e-03,
          -2.5444e-02, -3.8356e-02,  3.7833e-03,  1.9825e-02,  0.0000e+00,
          -5.1067e-02, -3.3640e-02,  4.5992e-02,  1.8659e-02,  5.5399e-03,
          -2.7899e-02, -1.0118e-02,  7.9805e-03,  4.8648e-02,  4.8376e-02,
           2.4903e-02, -3.3250e-02, -1.2652e-02,  7.0006e-03,  5.0891e-02,
           3.6230e-02,  1.3786e-02, -6.1919e-02,  3.0208e-02,  5.6953e-04,
          -2.4975e-03, -1.8701e-02,  1.7812e-03, -1.4314e-02,  1.0928e-02,
           1.6616e-02, -1.1800e-02,  2.0700e-02,  6.1527e-03, -1.7211e-03,
          -2.2236e-02, -5.0913e-02, -8.7408e-03,  5.7324e-02,  4.2661e-02,
          -3.1426e-04, -2.3402e-02, -1.2742e-02,  3.3155e-02, -2.1730e-03,
          -3.7434e-03,  2.2314e-02, -4.3151

Bad pipe message: %s [b'\x1f\xe67lQ\xa6\xde\x1f\xec\x1b0\x07\xedw\xf2\xf1`\x99 "\x0c?d4w\x1aN\xde#\x01\x16\x91\xd3\tv\x8c\xb2\xad!\xb6\x9d\xd9\xf0\x1e\xae\xfa\xcb4\xbd\x89m\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00', b'']
Bad pipe message: %s [b':\xdf\xf5\xe8\x1fAi\x0e\n\xe1P\x99\xca9^>3[ \xabw\xccD\xd7\xfc\xc3H\xb8{M\x14\xfe\xa3ul\x95\nv\x81\xd5\xce\x02\x12,M]\xafT\x9e\xde\xe4\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17']
Bad pipe message: %s [b'c\xa8\xd0l\x83\xb5\x01\x88\x1b\xab]\xeb\x7f\xcd\x9f\x7f(\xed\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(']
Bad pipe message: %s [b'^\x18', b"\xc7\xfd\x8a\xf8W\x80\xe5\xf32\x95\xd6\xb2\x19*\